# Capstone Project

In [28]:
import pandas as pd
import numpy as np

In [29]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# PART I. Scrap Data from Wikipedia

In [30]:
!pip install bs4

**Import libraries for scraping, initialize headers and URL**

In [31]:
import requests
from bs4 import BeautifulSoup
import lxml.etree as xml
import re

headers = {
    'authority': 'scrapeme.live',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

**Define Scraping function**

In [32]:
def _extractData(URL):
    try:
        response = requests.get(URL, headers=headers)

        web_page = BeautifulSoup(response.text, 'html5lib')
        table = web_page.find_all(name="table", attrs={"class": "wikitable"})[0]

        df = pd.read_html(str(table))[0]
    except Exception as e:
        print(e)
        
    return df


dataframe = _extractData(URL)

In [33]:
dataframe.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)
dataframe

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


# PART II. PreProcessing the Data

1. Deleting rows where Borough is "Not Assigned"
2. If "Neighborhood" is not assigned it takes the same value of "Borough"

**Ignore/Delete Rows where Borough is Not Assigned**

In [34]:
dataframe = dataframe.loc[ dataframe.Borough != "Not assigned"]
dataframe = dataframe.sort_values(by="Postal Code")
dataframe

,Postal Code,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
107,M9P,Etobicoke,Westmount
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


**Rows with Borough Not Assigned were taken off the dataframe**

In [35]:
dataframe.loc[ dataframe["Neighborhood"] == "Not assigned" ]

,Postal Code,Borough,Neighborhood


**All Neighbourhood are assigned to a Borough**

In [36]:
print("The shape of this Dataframe is : ", dataframe.shape)
print(r"With",dataframe.shape[0],"rows and", dataframe.shape[1],"columns")

The shape of this Dataframe is :  (103, 3)
With 103 rows and 3 columns


# PART III. GeoCoding

**Try with the geospatial datasheets**

In [37]:
## Download the geospatial datasheets
# !wget https://cocl.us/Geospatial_data

df_coords = pd.read_csv('Geospatial_data')
df_coords = df_coords.sort_values(by="Postal Code")
df_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


## Merging both the dataframes together

In [38]:
neighborhoods = dataframe.merge(df_coords)
neighborhoods

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# PART IV. Visualization

In [39]:
!pip install folium
!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab

In [40]:
from geopy.geocoders import Nominatim
import folium

**Get the latitude and longitude values of Toronto, CA & Map it**

In [41]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [42]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10, width=1000, height=500)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Initialize Foursquare Credentials

In [162]:
CLIENT_ID = 'K11QLHP35WUZC4RTZHMVWMLCFLGP4ILK1H4NPHEZB4OIEWLF' # your Foursquare ID
CLIENT_SECRET = '3MEIEWHMOACYUXQKFX5Y0UI51JLOYNHG2XM24LZZU342EIIU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K11QLHP35WUZC4RTZHMVWMLCFLGP4ILK1H4NPHEZB4OIEWLF
CLIENT_SECRET:3MEIEWHMOACYUXQKFX5Y0UI51JLOYNHG2XM24LZZU342EIIU


## Exploring the Neighborhoods

In [163]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [164]:
# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [165]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON


# filter columns (keep only name, categories, lat, long)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Downtown Toronto,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.653232,-79.385296
1,Nathan Phillips Square,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",43.652270,-79.383516
2,Indigo,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",43.653515,-79.380696
3,Chatime 日出茶太,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",43.655542,-79.384684
4,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",43.654447,-79.380952
...,...,...,...,...
67,Fine Asian Bowl,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",43.655387,-79.380326
68,Pantages Hotel & Spa,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",43.654498,-79.379035
69,Tim Hortons,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.655212,-79.380063
70,Pantages Lounge & Bar,"[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",43.654493,-79.379000


In [166]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
4,CF Toronto Eaton Centre,Shopping Mall,43.654447,-79.380952
...,...,...,...,...
67,Fine Asian Bowl,Vietnamese Restaurant,43.655387,-79.380326
68,Pantages Hotel & Spa,Hotel,43.654498,-79.379035
69,Tim Hortons,Coffee Shop,43.655212,-79.380063
70,Pantages Lounge & Bar,Cocktail Bar,43.654493,-79.379000


In [183]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
...,...,...,...,...,...,...,...
2122,"Northwest, West Humber - Clairville",43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location
2123,"Northwest, West Humber - Clairville",43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar
2124,"Northwest, West Humber - Clairville",43.706748,-79.594054,Saand Rexdale,43.705072,-79.598725,Drugstore
2125,"Northwest, West Humber - Clairville",43.706748,-79.594054,PC Garden,43.706539,-79.599359,Garden Center


In [80]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


## Analyse Neighborhood

In [208]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
neigh = toronto_onehot.pop('Neighborhood')
toronto_onehot.insert(0, 'Neighborhood', neigh)
# move neighborhood column to the first column
# fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
# toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122,"Northwest, West Humber - Clairville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2123,"Northwest, West Humber - Clairville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2124,"Northwest, West Humber - Clairville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2125,"Northwest, West Humber - Clairville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Cluster Labels,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.888889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.318182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.478261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",0.558824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0
92,"Willowdale, Willowdale West",1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
93,Woburn,1.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
94,Woodbine Heights,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


# Get Most Common Venues

In [214]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3             Clothing Store  0.25
4             Cluster Labels  0.00


----Alderwood, Long Branch----
                venue  freq
0      Cluster Labels  0.89
1         Pizza Place  0.22
2  Athletics & Sports  0.11
3         Coffee Shop  0.11
4                 Pub  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0  Cluster Labels  0.32
1     Coffee Shop  0.09
2            Bank  0.09
3     Pizza Place  0.05
4   Deli / Bodega  0.05


----Bayview Village----
                 venue  freq
0       Cluster Labels  1.00
1                 Café  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4  Japanese Restaurant  0.25


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Cluster Labels  0.48
1      Sandwich Place  0.09
2  Italian Restaurant  0.09
3         Cof

                  venue  freq
0        Cluster Labels  0.54
1  Fast Food Restaurant  0.15
2     Indian Restaurant  0.08
3           Coffee Shop  0.08
4    Chinese Restaurant  0.08


----Stn A PO Boxes----
                venue  freq
0      Cluster Labels  0.36
1         Coffee Shop  0.11
2  Seafood Restaurant  0.04
3  Italian Restaurant  0.03
4          Restaurant  0.03


----Studio District----
                 venue  freq
0       Cluster Labels  0.39
1          Coffee Shop  0.08
2  American Restaurant  0.06
3                 Café  0.06
4               Bakery  0.06


----Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park----
            venue  freq
0  Cluster Labels  0.50
1     Coffee Shop  0.14
2      Bagel Shop  0.07
3             Pub  0.07
4      Restaurant  0.07


----The Annex, North Midtown, Yorkville----
               venue  freq
0     Cluster Labels  1.00
1     Sandwich Place  0.16
2               Café  0.16
3        Coffee Shop  0.11
4  Indian Restaurant  0.05

In [215]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [216]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Clothing Store,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Cluster Labels,Pizza Place,Athletics & Sports,Gym,Coffee Shop,Pub,Sandwich Place,Skating Rink,Pharmacy,Airport Terminal
2,"Bathurst Manor, Wilson Heights, Downsview North",Cluster Labels,Bank,Coffee Shop,Park,Deli / Bodega,Ice Cream Shop,Sushi Restaurant,Middle Eastern Restaurant,Shopping Mall,Mobile Phone Shop
3,Bayview Village,Cluster Labels,Café,Japanese Restaurant,Bank,Chinese Restaurant,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Cluster Labels,Coffee Shop,Sandwich Place,Italian Restaurant,Thai Restaurant,Pharmacy,Pizza Place,Cupcake Shop,Pub,Café
...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",Cluster Labels,Ramen Restaurant,Café,Shopping Mall,Sandwich Place,Sushi Restaurant,Coffee Shop,Pizza Place,Movie Theater,Bank
92,"Willowdale, Willowdale West",Cluster Labels,Coffee Shop,Discount Store,Pharmacy,Pizza Place,Grocery Store,Farmers Market,Event Space,Ethiopian Restaurant,Escape Room
93,Woburn,Cluster Labels,Coffee Shop,Korean BBQ Restaurant,Ethiopian Restaurant,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Dessert Shop,Drugstore
94,Woodbine Heights,Skating Rink,Intersection,Park,Athletics & Sports,Beer Store,Curling Ice,Doner Restaurant,Diner,Discount Store,Distribution Center


## Cluster Neighborhoods

In [218]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 4, 1, 4, 4, 1, 1, 4, 2], dtype=int32)

In [219]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [220]:
toronto_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(inplace=True) # check the last columns!
toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Cluster Labels,Fast Food Restaurant,Grocery Store,Falafel Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Bar,Home Service,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dessert Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Mexican Restaurant,Electronics Store,Intersection,Restaurant,Breakfast Spot,Rental Car Location,Medical Center,Bank,Yoga Studio,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Cluster Labels,Coffee Shop,Korean BBQ Restaurant,Ethiopian Restaurant,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Dessert Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Fried Chicken Joint,Gas Station,Hakka Restaurant,Bakery,Lounge,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Bank,Diner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188,0.0,Park,Convenience Store,Jewelry Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,1.0,Cluster Labels,Discount Store,Intersection,Pizza Place,Sandwich Place,Coffee Shop,Middle Eastern Restaurant,Chinese Restaurant,Dim Sum Restaurant,Diner
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,1.0,Cluster Labels,Sandwich Place,Park,Bus Line,Pizza Place,Airport Food Court,Falafel Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,4.0,Cluster Labels,Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Sandwich Place,Beer Store,Japanese Restaurant,Fast Food Restaurant,Video Store


In [221]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11, width=1000, height=500)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters